**Data Loading**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
chatbot = pd.read_csv('chatbot_cleaned.csv')
print(chatbot.shape)
chatbot.head()

(80, 4)


,response_id,class,response_text,text_token
0,response_1,0,try avoid sort conflict,"['try', 'avoid', 'sort', 'conflict']"
1,response_2,1,friend open mental addiction weed taking life ...,"['friend', 'open', 'mental', 'addiction', 'wee..."
2,response_3,1,saved girl suicide going swallow bunch pills t...,"['saved', 'girl', 'suicide', 'going', 'swallow..."
3,response_4,0,cant think one really think may indirectly,"['cant', 'think', 'one', 'really', 'think', 'm..."
4,response_5,0,really one friend fit categories therapist cal...,"['really', 'one', 'friend', 'fit', 'categories..."


In [3]:
resume = pd.read_csv('resume_cleaned.csv')
print(resume.shape)
resume.head()

(125, 4)


,resume_id,class,resume_text,text_token
0,resume_1,0,customer service supervisor tier isabella cata...,"['customer', 'service', 'supervisor', 'tier', ..."
1,resume_2,0,engineer scientist ibm microelectronics divisi...,"['engineer', 'scientist', 'ibm', 'microelectro..."
2,resume_3,0,lts software engineer computational lithograph...,"['lts', 'software', 'engineer', 'computational..."
3,resume_4,0,tutor williston email indeed indeed com alec s...,"['tutor', 'williston', 'vt', 'email', 'indeed'..."
4,resume_5,1,independent consultant self employed burlingto...,"['independent', 'consultant', 'self', 'employe..."


**Encoding**

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
ohe = preprocessing.OneHotEncoder()
tfidf = TfidfVectorizer()
countvec = CountVectorizer()

In [5]:
chatbot_X = tfidf.fit_transform(chatbot['response_text'])
chatbot_y = chatbot['class']

resume_X = tfidf.fit_transform(resume['resume_text'])
resume_y = resume['class']

**Train/Test Splitting**

In [6]:
from sklearn.model_selection import train_test_split

chatbot_X_train, chatbot_X_test, chatbot_y_train, chatbot_y_test = train_test_split(chatbot_X, chatbot_y, test_size=0.2, random_state=41)
resume_X_train, resume_X_test, resume_y_train, resume_y_test = train_test_split(resume_X, resume_y, test_size=0.2, random_state=41)

chatbot_X_train = chatbot_X_train.toarray()
chatbot_X_test = chatbot_X_test.toarray()
resume_X_train = resume_X_train.toarray()
resume_X_test = resume_X_test.toarray()


In [7]:
import tensorflow as tf
import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, SimpleRNN, Embedding

2024-04-21 06:01:28.431411: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 06:01:28.454191: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 06:01:28.454970: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 06:01:29.009879: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


**Modeling**

*GRU*

Simple GRU (No Dropout)

In [8]:
simple_GRU_model = Sequential()
simple_GRU_model.add(Embedding(1000, 32))
simple_GRU_model.add(GRU(100))
simple_GRU_model.add(Dense(1, activation='sigmoid'))

simple_GRU_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

simple_GRU_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 gru (GRU)                   (None, 100)               40200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 72301 (282.43 KB)
Trainable params: 72301 (282.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-04-21 06:01:29.583935: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-21 06:01:29.584218: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
print("Chatbot Simple GRU Accuracy Results:")
simple_GRU_model.fit(chatbot_X_train, chatbot_y_train, epochs=5, batch_size=32, validation_data=(chatbot_X_test, chatbot_y_test))

Chatbot Simple GRU Accuracy Results:
Epoch 1/5
2/2 [==============================] - 1s 475ms/step - loss: 0.6845 - accuracy: 0.6875 - val_loss: 0.6760 - val_accuracy: 0.6875
Epoch 2/5
2/2 [==============================] - 0s 164ms/step - loss: 0.6733 - accuracy: 0.6875 - val_loss: 0.6628 - val_accuracy: 0.6875
Epoch 3/5
2/2 [==============================] - 0s 135ms/step - loss: 0.6603 - accuracy: 0.6875 - val_loss: 0.6500 - val_accuracy: 0.6875
Epoch 4/5
2/2 [==============================] - 0s 140ms/step - loss: 0.6461 - accuracy: 0.6875 - val_loss: 0.6380 - val_accuracy: 0.6875
Epoch 5/5
2/2 [==============================] - 0s 133ms/step - loss: 0.6352 - accuracy: 0.6875 - val_loss: 0.6274 - val_accuracy: 0.6875


In [10]:
print("Resume Simple GRU Accuracy Results:")
simple_GRU_model.fit(resume_X_train, resume_y_train, epochs=5, batch_size=32, validation_data=(resume_X_test, resume_y_test))

Resume Simple GRU Accuracy Results:
Epoch 1/5
4/4 [==============================] - 26s 6s/step - loss: 0.6099 - accuracy: 0.7100 - val_loss: 0.4783 - val_accuracy: 0.8400
Epoch 2/5
4/4 [==============================] - 25s 6s/step - loss: 0.6149 - accuracy: 0.7100 - val_loss: 0.4474 - val_accuracy: 0.8400
Epoch 3/5
4/4 [==============================] - 25s 6s/step - loss: 0.6299 - accuracy: 0.7100 - val_loss: 0.4439 - val_accuracy: 0.8400
Epoch 4/5
4/4 [==============================] - 24s 6s/step - loss: 0.6274 - accuracy: 0.7100 - val_loss: 0.4494 - val_accuracy: 0.8400
Epoch 5/5
4/4 [==============================] - 24s 6s/step - loss: 0.6187 - accuracy: 0.7100 - val_loss: 0.4596 - val_accuracy: 0.8400


In [11]:
dropout_GRU_model = Sequential()
dropout_GRU_model.add(Embedding(5000, 32))
dropout_GRU_model.add(SimpleRNN(100, dropout=0.2, recurrent_dropout=0.2))
dropout_GRU_model.add(Dense(1, activation='sigmoid'))

dropout_GRU_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

dropout_GRU_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          160000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               13300     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 173401 (677.35 KB)
Trainable params: 173401 (677.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
print("Chatbot Dropout GRU Accuracy Results:")
dropout_GRU_model.fit(chatbot_X_train, chatbot_y_train, epochs=5, batch_size=32, validation_data=(chatbot_X_test, chatbot_y_test))

Chatbot Dropout GRU Accuracy Results:
Epoch 1/5
2/2 [==============================] - 1s 176ms/step - loss: 0.7120 - accuracy: 0.5156 - val_loss: 0.7074 - val_accuracy: 0.3125
Epoch 2/5
2/2 [==============================] - 0s 79ms/step - loss: 0.6889 - accuracy: 0.5781 - val_loss: 0.7113 - val_accuracy: 0.3125
Epoch 3/5
2/2 [==============================] - 0s 70ms/step - loss: 0.7577 - accuracy: 0.4844 - val_loss: 0.7120 - val_accuracy: 0.3125
Epoch 4/5
2/2 [==============================] - 0s 69ms/step - loss: 0.7315 - accuracy: 0.4219 - val_loss: 0.7103 - val_accuracy: 0.3125
Epoch 5/5
2/2 [==============================] - 0s 84ms/step - loss: 0.7139 - accuracy: 0.5312 - val_loss: 0.7062 - val_accuracy: 0.3125


In [13]:
print("Resume Dropout GRU Accuracy Results:")
dropout_GRU_model.fit(resume_X_train, resume_y_train, epochs=5, batch_size=32, validation_data=(resume_X_test, resume_y_test))

Resume Dropout GRU Accuracy Results:
Epoch 1/5
4/4 [==============================] - 21s 5s/step - loss: 0.6899 - accuracy: 0.5400 - val_loss: 0.6967 - val_accuracy: 0.1600
Epoch 2/5
4/4 [==============================] - 21s 5s/step - loss: 0.6930 - accuracy: 0.5500 - val_loss: 0.6778 - val_accuracy: 0.8400
Epoch 3/5
4/4 [==============================] - 20s 5s/step - loss: 0.6864 - accuracy: 0.5400 - val_loss: 0.6602 - val_accuracy: 0.8400
Epoch 4/5
4/4 [==============================] - 20s 5s/step - loss: 0.6958 - accuracy: 0.5400 - val_loss: 0.6422 - val_accuracy: 0.8400
Epoch 5/5
4/4 [==============================] - 19s 5s/step - loss: 0.6743 - accuracy: 0.5600 - val_loss: 0.6251 - val_accuracy: 0.8400


: 